<a href="https://colab.research.google.com/github/sangmin213/DialectClassification/blob/main/model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import os
import re
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import pickle
from tqdm import tqdm
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
from torch.utils.data import Subset

In [ ]:
index2region={0:'gangwon', 1:'gyeongsang', 2:'jeonla', 3:'chungcheong', 4:'jeju'}
region2index = {v:k for k,v in index2region.items()}
region_shortening = ['GW','GS','JL','CC','JJ']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 데이터 셋 구성 (full dataset)
dataset_dir = '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/'

In [ ]:
region_dir = glob(dataset_dir)
region_dir

['/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/']

In [ ]:
for k, v in index2region.items():
    exec(f"{v}_dirs = glob(dataset_dir+'*_{v}/*')")
jeonla_dirs

['/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000018',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000027',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000012',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000032',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000005',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000014',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000006',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000019',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000024',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000015']

In [ ]:
def make_tuple_data(dirs, max_num):
    for i, region_dir in enumerate(dirs):
        if i>=max_num:break
        spectro_path = glob(region_dir+'/*_spectro.pickle')[0]
        mfcc_path = glob(region_dir+'/*_mfcc.pickle')[0]
        chroma_path = glob(region_dir+'/*_chroma.pickle')[0]
        
        with open(spectro_path, "rb") as f:
            spectro = pickle.load(f)
        with open(mfcc_path, "rb") as f:
            mfcc = pickle.load(f)
        with open(chroma_path, "rb") as f:
            chroma = pickle.load(f)

        if i == 0:
            spectro_data = spectro
            mfcc_data = mfcc
            chroma_data = chroma
        else:
            spectro_data = np.concatenate([spectro_data,spectro], axis=0)
            mfcc_data = np.concatenate([mfcc_data,mfcc], axis=0)
            chroma_data = np.concatenate([chroma_data,chroma], axis=0)
    if max_num ==0:return []
        
    r_data = [(s,m,c) for s,m,c in zip(spectro_data,mfcc_data,chroma_data)]
        
    return r_data

def make_tuple(max_num=2):
    jeonla_data = make_tuple_data(jeonla_dirs, max_num)
    chungcheong_data = make_tuple_data(chungcheong_dirs, max_num)
    gyeongsang_data = make_tuple_data(gyeongsang_dirs, max_num)
    jeju_data = make_tuple_data(jeju_dirs, max_num)
    gangwon_data = make_tuple_data(gangwon_dirs, max_num)
    return jeonla_data, chungcheong_data, gyeongsang_data, jeju_data, gangwon_data

def print_data(r_data, region):
    if len(r_data)==0: return
    print(f"{region} data num: ", len(r_data))
    print(f"{region} tuple size", len(r_data[0]))
    print(f"{region} spec shape", r_data[0][0].shape)

jeonla_data, chungcheong_data, gyeongsang_data, jeju_data, gangwon_data = make_tuple(1000)
print_data(jeonla_data, 'jeonla')
print_data(chungcheong_data, 'chungcheong')
print_data(gyeongsang_data, 'gyeongsang')
print_data(jeju_data, 'jeju')
print_data(gangwon_data, 'gangwon')

jeonla data num:  913
jeonla tuple size 3
jeonla spec shape (201, 501)
chungcheong data num:  1108
chungcheong tuple size 3
chungcheong spec shape (201, 501)
gyeongsang data num:  842
gyeongsang tuple size 3
gyeongsang spec shape (201, 501)
jeju data num:  756
jeju tuple size 3
jeju spec shape (201, 501)
gangwon data num:  1183
gangwon tuple size 3
gangwon spec shape (201, 501)


In [ ]:
jeonla_data_l = []
for data in jeonla_data:
    y = [0,0,0,0,0]
    y[region2index['jeonla']] = 1
    jeonla_data_l.append((data,y))

chungcheong_data_l = []
for data in chungcheong_data:
    y = [0,0,0,0,0]
    y[region2index['chungcheong']] = 1
    chungcheong_data_l.append((data,y))

gyeongsang_data_l = []
for data in gyeongsang_data:
    y = [0,0,0,0,0]
    y[region2index['gyeongsang']] = 1
    gyeongsang_data_l.append((data,y))

jeju_data_l = []
for data in jeju_data:
    y = [0,0,0,0,0]
    y[region2index['jeju']] = 1
    jeju_data_l.append((data,y))

gangwon_data_l = []
for data in gangwon_data:
    y = [0,0,0,0,0]
    y[region2index['gangwon']] = 1
    gangwon_data_l.append((data,y))

In [ ]:
datasumup = np.concatenate([jeonla_data_l[:500], chungcheong_data_l[:500], gangwon_data_l[:500], jeju_data_l[:500], gyeongsang_data_l[:500]], axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
from sklearn.preprocessing import normalize
class MultiModalDataset(Dataset):

    def __init__(self, data):

        self.data = data

    def __getitem__(self, idx):
        datas, label = self.data[idx]
        spec, mfcc, chroma = datas
        spec, mfcc, chroma = normalize(spec), normalize(mfcc), normalize(chroma)
        spec, mfcc, chroma = torch.tensor(spec, dtype=torch.float32), torch.tensor(mfcc, dtype=torch.float32), torch.tensor(chroma, dtype=torch.float32)
        spec, mfcc, chroma = spec.unsqueeze(0), mfcc.unsqueeze(0), chroma.unsqueeze(0)
        label = torch.tensor(label, dtype=torch.float32)

        data = (spec, mfcc, chroma)
        return data, label
    
    def __len__(self):
        return len(self.data)

In [ ]:
dataset = MultiModalDataset(datasumup)
len(dataset)

2500

# ResNet

In [ ]:
class BasicBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.seq1 = nn.Sequential(self.conv1, self.bn1, self.relu)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(3,3), stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.seq2 = nn.Sequential(self.conv2, self.bn2)
        
        self.down_flag = False
        if in_channels != out_channels: self.down_flag = True

        self.downsample = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1,1), stride=2, padding=0, bias=False)
    
    def forward(self, x):
        #print(x.shape)
        y = self.seq1(x)
        #print(y.shape)
        y = self.seq2(y)
        #print(y.shape)

        if self.down_flag:
            x = self.downsample(x)
        
        y = self.relu(y)
        #print(x.shape)
        #print(y.shape)
        y = y + x

        return y
        

In [ ]:
class ResNet18(nn.Module):

    def __init__(self, in_channels, output_dim=256, model_type='spec',best_model_save_path="./ResNet_best_model.pt"):
        super(ResNet18, self).__init__()

        self.best_model_save_path = best_model_save_path
        self.data_type = model_type

        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=(7,7), stride=2, padding=3)
        self.BN1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)

        self.seq1 = nn.Sequential(self.conv1, self.BN1, self.pool1)

        self.seq2 = nn.Sequential(BasicBlock(64,64), BasicBlock(64,64))
        self.seq3 = nn.Sequential(BasicBlock(64,64), BasicBlock(64, 128, stride=2))
        self.seq4 = nn.Sequential(BasicBlock(128,128), BasicBlock(128,128))
        self.seq5 = nn.Sequential(BasicBlock(128,128), BasicBlock(128,256,stride=2))

        self.avg_pool1 = nn.AdaptiveAvgPool2d((1,1))
        # if model_type=='spec':
        #     self.fc1 = nn.Linear(256*13*32, output_dim)
        # elif model_type=='mfcc':
        #     self.fc1 = nn.Linear(256*7*32, output_dim)
        # elif model_type=='chroma':
        #     self.fc1 = nn.Linear(256*1*32, output_dim)
        self.fc1 = nn.Linear(256, output_dim)


        self.lastlayer = nn.Sequential(self.fc1, self.relu)

    def forward(self, x):
        y = self.seq1(x)
        y = self.seq2(y)
        y = self.seq3(y)
        y = self.seq4(y)
        y = self.seq5(y)
        y = self.avg_pool1(y)
        y = y.view(y.shape[0],-1)
        y = self.lastlayer(y)

        return y

    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        self.pred_labels_train = []
        self.real_labels_train = []
        self.pred_labels_val = None
        self.real_labels_val = None

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_f=nn.CrossEntropyLoss()

        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data

                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data.shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                
                if epoch==epochs: #last epoch
                    self.pred_labels_train.append(pred_indices)
                    self.real_labels_train.append(batch_label)
                #for p, l in zip(pred_indices, batch_label):
                #    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")
            

            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            if epoch==epochs: #last epoch
                self.pred_labels_val=predicted.cpu().numpy()
                self.real_labels_val=labels.cpu().numpy()
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")
            print(f"epoch {epoch} val loss: {val_loss}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
        
        self.pred_labels_train = torch.cat(self.pred_labels_train, dim=0)
        self.real_labels_train = torch.cat(self.real_labels_train, dim=0)
        self.pred_labels_train = self.pred_labels_train.cpu().numpy()
        self.real_labels_train = self.real_labels_train.cpu().numpy()
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss_f(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss
        elif which=='confusion_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening)
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening, normalize='true')
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return        
        elif which=='confusion_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening)
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening, normalize='true')
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return   
            

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
        plt.show()

# LeNet

In [ ]:
class Block(nn.Module):
    def __init__(self,input_channel, output_channel, kernel_size, stride=1, padding=0):
        super(Block,self).__init__()

        self.conv1=nn.Conv2d(input_channel,output_channel,kernel_size=kernel_size,padding=padding) # no stride
        self.conv2=nn.Conv2d(output_channel,output_channel,kernel_size=kernel_size,stride=stride,padding=padding) # stride if down sampling
        self.bn=nn.BatchNorm2d(output_channel)
        self.relu=nn.ReLU()

        self.layer=nn.Sequential(self.conv1, self.bn, self.relu, self.conv2, self.bn)

        self.stride=stride
        self.iden = nn.Conv2d(input_channel, output_channel, kernel_size=(1,1), stride=1)
        if input_channel==64:
            self.iden = nn.Conv2d(input_channel, output_channel, kernel_size=(1,1), stride=1)

    def forward(self,x):
        y = self.layer(x)
        if self.stride==1: # stride==2 인 경우는 downsampling 구간이기 때문에 residual 안함
            y = y + self.iden(x)
        y = self.relu(y)

        return y

In [ ]:
class LeNet(nn.Module):
    def __init__(self,data_type="mfcc",method="origin",best_model_save_path="./LeNet_best_model.pt"):
        super(LeNet,self).__init__()
        
        self.best_model_save_path = best_model_save_path
        self.data_type=data_type
        self.method=method

        if data_type=="mfcc": # (1,100,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(6,7),stride=2,padding=3)
        elif data_type=="spec": #(1,201,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(7,7),stride=2,padding=3)
        elif data_type=="chroma": #(1,12,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(6,7),stride=1,padding=3)
        self.maxpool=nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.avgpool=nn.AdaptiveAvgPool2d((1,1)) # global avg pool
        self.relu=nn.ReLU()

        self.seq1=nn.Sequential(self.conv1,self.relu,self.maxpool)  # (1,100,501)/(1,12,501)/(1,201,501) -> (64,51,251)/(1,13,251)/(1,101,251).conv -> (64,25,126)/(64,7,126)/(64,51,126).maxpool
        self.seq2=nn.Sequential(Block(64,64,(3,3),padding=1),Block(64,64,(3,3),padding=1),Block(64,128,(3,4),stride=2,padding=1)) # (64,25,126) -> (64,25,126).block -> (64,25,126).block -> (128,13,63)/(128,4,63)/(128,26,63).block 
        if data_type=="mfcc":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(3,3),stride=2,padding=1)) # (128,13,63) -> (128,13,63).block -> (128,13,63).block -> (256,7,32).block 
        elif data_type=="spec":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(4,3),stride=2,padding=1)) # (128,26,63) -> ... -> (256,12,32).block 
        elif data_type=="chroma":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(2,3),stride=2,padding=1)) # (128,4,63)/(128,26,63) -> ... -> (256,2,32)/(256,13,32).block 
        self.seq4=nn.Sequential(Block(256,256,(3,3),padding=1),Block(256,256,(3,3),padding=1),self.avgpool) # ... -> (256,1,1).avgpool

        if self.method=="multimodal":
            self.fc=nn.Linear(256,256)
        if self.method=="origin":
            self.fc=nn.Linear(256,5)
            self.loss=nn.CrossEntropyLoss()
            self.optimizer=optim.Adam(self.parameters(),lr=0.0001)

        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []

    def forward(self,x):       
        y=self.seq1(x)
        y=self.seq2(y)
        y=self.seq3(y)
        y=self.seq4(y)
        y=y.view(y.shape[0],-1)
        y=self.fc(y)
        return y

    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        self.pred_labels_train = []
        self.real_labels_train = []
        self.pred_labels_val = None
        self.real_labels_val = None

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_f=nn.CrossEntropyLoss()

        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data

                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data.shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                
                if epoch==epochs: #last epoch
                    self.pred_labels_train.append(pred_indices)
                    self.real_labels_train.append(batch_label)
                #for p, l in zip(pred_indices, batch_label):
                #    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")
            

            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            if epoch==epochs: #last epoch
                self.pred_labels_val=predicted.cpu().numpy()
                self.real_labels_val=labels.cpu().numpy()
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")
            print(f"epoch {epoch} val loss: {val_loss}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
        
        self.pred_labels_train = torch.cat(self.pred_labels_train, dim=0)
        self.real_labels_train = torch.cat(self.real_labels_train, dim=0)
        self.pred_labels_train = self.pred_labels_train.cpu().numpy()
        self.real_labels_train = self.real_labels_train.cpu().numpy()
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss_f(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss
        elif which=='confusion_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening)
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening, normalize='true')
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return        
        elif which=='confusion_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening)
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening, normalize='true')
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return   
            

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
        plt.show()

# LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self,data_type="mfcc",method="origin",best_model_save_path="./LSTM_best_model.pt"):
        super(LSTM,self).__init__()

        self.best_model_save_path=best_model_save_path
        self.data_type=data_type
        self.method=method

        self.hidden_size=64
        self.num_layers=1
        if self.data_type=="mfcc":
            self.input_size=100  #mfcc 기준
        elif self.data_type=="spec":
            self.input_size=201  #mfcc 기준
        elif self.data_type=="chroma":
            self.input_size=12  #mfcc 기준

        self.lstm=nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_size, num_layers =self.num_layers,batch_first=True)
        
        if self.method=="multimodal":
            self.linear=nn.Linear(32064,256) # 밑에 forward에서 y.reshape 부분 보면 이해됨.  hidden -> 256 output vector for multimodal 
        elif self.method=="origin":
            self.linear=nn.Linear(32064,5) # hidden -> 지역 개수

    def forward(self,x):
        x = x.reshape(x.shape[0],-1,x.shape[3]) # 32,100,501 = (batch size, 100, 501)
        x = x.view(x.shape[0],x.shape[2],-1) # 32,501,100 = (batch size, sequence len, input size)
        
        #Initial hidden state
        h_0 = torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(device=device)
        #Initial cell state
        c_0 = torch.zeros((self.num_layers, x.shape[0], self.hidden_size)).to(device=device)
        
        y, (h_n,c_n) =self.lstm(x,(h_0,c_0))
        y = y.reshape(y.shape[0],-1) # y = (batch size, sequence len * hidden_size) = (32, 501*64) 
        y = self.linear(y)

        return y

    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        self.pred_labels_train = []
        self.real_labels_train = []
        self.pred_labels_val = None
        self.real_labels_val = None

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_f=nn.CrossEntropyLoss()

        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data

                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data.shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                
                if epoch==epochs: #last epoch
                    self.pred_labels_train.append(pred_indices)
                    self.real_labels_train.append(batch_label)
                #for p, l in zip(pred_indices, batch_label):
                #    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")
            

            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            if epoch==epochs: #last epoch
                self.pred_labels_val=predicted.cpu().numpy()
                self.real_labels_val=labels.cpu().numpy()
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")
            print(f"epoch {epoch} val loss: {val_loss}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
        
        self.pred_labels_train = torch.cat(self.pred_labels_train, dim=0)
        self.real_labels_train = torch.cat(self.real_labels_train, dim=0)
        self.pred_labels_train = self.pred_labels_train.cpu().numpy()
        self.real_labels_train = self.real_labels_train.cpu().numpy()
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss_f(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss
        elif which=='confusion_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening)
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening, normalize='true')
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return        
        elif which=='confusion_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening)
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening, normalize='true')
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return   
            

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
        plt.show()



# Multimodal

In [ ]:
class MultiModalDialectClassifier(nn.Module):
    
    def __init__(self, hidden_dim=1024, out_dim=5, method = "ResNet", best_model_save_path="./best_model.pt"):
        super(MultiModalDialectClassifier, self).__init__()

        self.method = method
        self.best_model_save_path = best_model_save_path
        
        if self.method=="ResNet"
            self.spec_NN = ResNet18(1, model_type='spec')
            self.mfcc_NN = ResNet18(1, model_type='mfcc')
            self.chroma_NN = ResNet18(1, model_type='chroma')

        self.relu = nn.ReLU()

        self.fc1_resnet = nn.Linear(128*3, 128)
        self.fc2_resnet = nn.Linear(128,out_dim)
        self.lastlayer_resnet = nn.Sequential(self.fc1_resnet, self.relu, self.fc2_resnet)
        
        self.spec_lenet = LeNet(method="multimodal",data_type="spec")
        self.mfcc_lenet = LeNet(method="multimodal",data_type="mfcc")
        self.chroma_lenet = LeNet(method="multimodal",data_type="chroma")

        self.fc1_lenet = nn.Linear(5*3, 10)
        self.fc2_lenet = nn.Linear(10,out_dim)
        self.lastlayer_lenet = nn.Sequential(self.fc1_lenet, self.relu, self.fc2_lenet)

        self.loss_f = nn.CrossEntropyLoss()
        self.optimizer = None
        #self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        spec_x, mfcc_x, chroma_x = x

        if self.method == "ResNet":
            spec_y = self.spec_resnet(spec_x)
            mfcc_y = self.mfcc_resnet(mfcc_x)
            chroma_y = self.chroma_resnet(chroma_x)
            y = torch.cat([spec_y, mfcc_y, chroma_y], dim=1)
            y = y.view(y.shape[0], -1)
        
            y = self.lastlayer_resnet(y)

        if self.method == "LeNet":
            spec_y = self.spec_lenet(spec_x)
            mfcc_y = self.mfcc_lenet(mfcc_x)
            chroma_y = self.chroma_lenet(chroma_x)

            y = torch.cat([spec_y, mfcc_y, chroma_y], dim=1)
            y = y.view(y.shape[0], -1)
        
            y = self.lastlayer_lenet(y)

        #y = self.softmax(y)
        #print(y.shape)
        return y
    
    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        self.pred_labels_train = []
        self.real_labels_train = []
        self.pred_labels_val = None
        self.real_labels_val = None

        self.optimizer = optim.AdamW(self.parameters(), lr=learning_rate)


        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data
                spec, mfcc, chroma = spec.to(device), mfcc.to(device), chroma.to(device)
                batch_data = (spec, mfcc, chroma)
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data[0].shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                
                if epoch==epochs: #last epoch
                    self.pred_labels_train.append(pred_indices)
                    self.real_labels_train.append(batch_label)
                #for p, l in zip(pred_indices, batch_label):
                #    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")
            

            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            if epoch==epochs: #last epoch
                self.pred_labels_val=predicted.cpu().numpy()
                self.real_labels_val=labels.cpu().numpy()
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")
            print(f"epoch {epoch} val loss: {val_loss}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
        
        self.pred_labels_train = torch.cat(self.pred_labels_train, dim=0)
        self.real_labels_train = torch.cat(self.real_labels_train, dim=0)
        self.pred_labels_train = self.pred_labels_train.cpu().numpy()
        self.real_labels_train = self.real_labels_train.cpu().numpy()
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                spec, mfcc, chroma = spec.to(device), mfcc.to(device), chroma.to(device)
                batch_data = (spec, mfcc, chroma)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss_f(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss
        elif which=='confusion_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening)
            plt.title('train confusion matrix')
            plt.savefig(f"./result/model_{which}.png")
            plt.show()
            return
        elif which=='confusion_normalize_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening, normalize='true')
            plt.title('train confusion matrix')
            plt.savefig(f"./result/model_{which}.png")
            plt.show()    
            return        
        elif which=='confusion_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening)
            plt.title('val confusion matrix')
            plt.savefig(f"./result/model_{which}.png")
            plt.show()
            return
        elif which=='confusion_normalize_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening, normalize='true')
            plt.title('val confusion matrix')
            plt.savefig(f"./result/model_{which}.png")
            plt.show()    
            return   
            

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./result/model_{which}.png")
        plt.show()

    def getConvLayers(self, lenet):
        weights = []
        conv_layers = []
        for i in range(len(lenet)):
            if type(lenet[i]) == nn.Conv2d:
                weights.append(lenet[i].weight)
                conv_layers.append(lenet[i])
            elif type(lenet[i]) == nn.Sequential:
                for basic in lenet[i].children(): # basic block
                    for in_basic in basic.children():
                        if type(in_basic) == nn.Conv2d:
                            weights.append(in_basic.weight)
                            conv_layers.append(in_basic)
                        if type(in_basic) == nn.Sequential:
                            for in_basic_in_sequential in in_basic:
                                if type(in_basic_in_sequential) == nn.Conv2d:
                                    weights.append(in_basic_in_sequential.weight)
                                    conv_layers.append(in_basic_in_sequential)            
        return weights, conv_layers

    def extractConvLayer(self):
        children_ = list(self.children())
        spec_lenet = list(children_[0].children())
        mfcc_lenet = list(children_[1].children())
        chroma_lenet = list(children_[2].children()) 

        spec_weights, spec_layers = self.getConvLayers(spec_lenet)
        mfcc_weights, mfcc_layers = self.getConvLayers(mfcc_lenet)
        chroma_weights, chroma_layers = self.getConvLayers(chroma_lenet)

        self.spec_weights = spec_weights
        self.spec_layers = spec_layers
        self.mfcc_weights = mfcc_weights
        self.mfcc_layers = mfcc_layers
        self.chroma_weights = chroma_weights
        self.chroma_layers = chroma_layers
        
        return spec_weights, spec_layers, mfcc_weights, mfcc_layers, chroma_weights, chroma_layers

    def plotFilter(self, where='first', data_type='spec', when='before_train'):
        if data_type == 'spec':
            filters = self.spec_weights
        elif data_type == 'mfcc':
            filters = self.mfcc_weights
        elif data_type == 'chroma':
            filters = self.chroma_weights
        x_len = 0
        y_len = 0
        if where=='first': # 64x1x7x7
            filters = filters[0]
            filters = filters[:,0,:,:]
            plt.figure(figsize=(20,17))
            x_len=8
            y_len=8
        elif where=='middle': # 128x64x3x3
            filters = filters[16]
            filters = filters[:,0,:,:]
            plt.figure(figsize=(30,25))
            x_len= 16
            y_len= 8
        elif where=='last': # 256x256x3x3
            filters = filters[len(filters)-2]
            filters = filters[:,0,:,:]
            plt.figure(figsize=(40,32))
            x_len=16
            y_len=16
        for i,filter in enumerate(filters):
            plt.subplot(x_len, y_len, i+1)
            plt.imshow(filter.detach().cpu(), cmap='gray')
            plt.axis('off')
        plt.savefig(f"./result/{data_type}_filter_{where}_{when}.png")
        plt.show()
        plt.close()
    
    def plotOriginalImage(self, data):
        plt.imshow(data[0,:,:])
        plt.show()

    def plotFeatureMap(self, data, where='first', data_type='spec', when='before_train'):
        if data_type == 'spec':
            layers = self.spec_layers
            x_len = 8
            y_len = 8
        elif data_type == 'mfcc':
            layers = self.mfcc_layers
            x_len = 16
            y_len = 4
        elif data_type == 'chroma':
            layers = self.chroma_layers
            x_len = 32
            y_len = 2

        if where=='first':
            plt.figure(figsize=(20,17))
            layer = layers[0]
        elif where=='middle':
            pass
        elif where=='last':
            pass
        results = layer(data.to(device)) # 64x?x251
        for i, result in enumerate(results):
            plt.subplot(x_len, y_len, i+1)
            plt.imshow(result.detach().cpu())
            plt.axis('off')
        plt.savefig(f"./result/{data_type}_feature_map_{where}_{when}.png")
        plt.show()
        plt.close()      

        
        
        

# Cross Validation

In [ ]:
def CV_Plot(title, arg, y):
        X = [i for i in range(1, len(y) + 1)]
        plt.xlabel("epoch")
        plt.ylabel(title)
        plt.title(title)
        plt.plot(X, y, label=title)
        plt.savefig(f"./result/model_{title}_{arg}.png")
        plt.show()

In [ ]:
def CrossValidation(dataset, learning_rate, epochs, device, method = "ResNet",data_type="mfcc"):
        hparams = []
        for i in range(len(learning_rate)):
            for j in range(len(epochs)):
                hparams.append((learning_rate[i], epochs[j]))
        print(hparams)

        train_dataset_l = []
        validation_dataset_l = []

        kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

        for train_index, test_index in kf.split(train_dataset):
            train_dataset_l.append(Subset(train_dataset,train_index))
            validation_dataset_l.append(Subset(train_dataset,test_index))

        result = []
        for i in range(len(hparams)):
            lr = hparams[i][0]
            e = hparams[i][1]

            print(f"Learning rate : {lr}, Epochs : {e}")

            last_val_acc = []
            for j in range(5):
                print(f"#{j+1} validation")
                if method=="Multimodal_ResNet"
                    model = MultiModalDialectClassifier(method="ResNet").to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="Multimodal_LeNet"
                    model = MultiModalDialectClassifier(method="LeNet").to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="ResNet"
                    model = ResNet18(1,output_dim=5,model_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="LeNet"
                    model = LeNet(data_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="LSTM"
                    model = LSTM(data_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                train_loader = DataLoader(train_dataset_l[j], batch_size=32, shuffle=True)
                validation_loader = DataLoader(validation_dataset_l[j], batch_size=32, shuffle=True)

                model.train_(train_loader, validation_loader, lr, e, device)
                last_val_acc.append(model.val_accuracy[-1])
                    
                # model.plot('train_acc')
                # model.plot('val_acc')
            result.append((np.array(last_val_acc)).mean())

        idx = result.index(max(result))
        best_lr, best_ep = hparams[idx]
        print(f"Best Learning Rate : {best_lr}, Best Epoch : {best_ep}")

        return best_lr, best_ep

In [ ]:
train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

사용법

1. CrossValidation() 함수를 사용하여 테스트한다. 이 때 결정할 인자는 아래와 같다.
2. method를 다음 중 결정한다. {"ResNet","LeNet","LSTM","Multimodal_ResNet","Multimodal_LeNet"}

3. dat_type을 다음 중 결정한다. {"spec","mfcc","chroma"} -> multimodal은 모든 데이터를 사용하게끔 설계되어 있으므로 안 적어도 되고, 아무 데이터 타입을 적어도 상관없다.

In [ ]:
lr = [0.0001, 0.0005, 0.001]
ep = [30]

result = CrossValidation(train_dataset, learning_rate= lr, epochs= ep, device = device, method = "ResNet",data_type="mfcc")

[(0.001, 15), (0.0005, 15), (0.0001, 15)]
Learning rate : 0.0001, Epochs : 15
#1 validation


100%|██████████| 50/50 [00:19<00:00,  2.56it/s]


epoch 1 time: 19.515663146972656sec(s).
epoch 1 train accuracy: 0.69875
epoch 1 loss: 0.951231119632721


100%|██████████| 13/13 [00:01<00:00,  7.55it/s]


epoch 1 val accuracy: 0.255
epoch 1 val loss: 2.683886326276339


...

epoch 14 val accuracy: 0.705
epoch 14 val loss: 1.2387986091467051


100%|██████████| 50/50 [00:19<00:00,  2.57it/s]


epoch 15 time: 19.50408148765564sec(s).
epoch 15 train accuracy: 0.995
epoch 15 loss: 0.02490442062728107


100%|██████████| 13/13 [00:01<00:00,  7.53it/s]

epoch 15 val accuracy: 0.405
epoch 15 val loss: 3.84400252195505
Finish!


Cross Val result

In [ ]:
print(result)

# Model Train & Test(feat. Plot)

사용법

1. model_choose() 함수를 사용하여 model을 결정한다. 이 때 고려할 인자는 아래와 같다.
2. method를 다음 중 결정한다. {"ResNet","LeNet","LSTM","Multimodal_ResNet","Multimodal_LeNet"}

3. dat_type을 다음 중 결정한다. {"spec","mfcc","chroma"} -> multimodal은 모든 데이터를 사용하게끔 설계되어 있으므로 안 적어도 되고, 아무 데이터 타입을 적어도 상관없다.

In [ ]:
model=None

def model_choose(model,method,data_type):
    if method=="Multimodal_ResNet"
        model = MultiModalDialectClassifier(method="ResNet").to(device) 
    elif method=="Multimodal"
        model = MultiModalDialectClassifier(method="LeNet").to(device) 
    elif method=="ResNet"
        model = ResNet18(1,output_dim=5,model_type=data_type).to(device) 
    elif method=="LeNet"
        model = LeNet(data_type=data_type).to(device) 
    elif method=="LSTM"
        model = LSTM(data_type=data_type).to(device) 

model_choose(model,"Multimodal_ResNet","mfcc")

spec_weights, spec_layers, mfcc_weights, mfcc_layers, chroma_weights, chroma_layers = model.extractConvLayer()

In [ ]:
model.plotOriginalImage(dataset[0][0][0])

In [ ]:
model.plotFilter(where='first', data_type='spec', when='before_train')
model.plotFilter(where='middle', data_type='spec', when='before_train')
model.plotFilter(where='last', data_type='spec', when='before_train')
model.plotFilter(where='first', data_type='mfcc', when='before_train')
model.plotFilter(where='middle', data_type='mfcc', when='before_train')
model.plotFilter(where='last', data_type='mfcc', when='before_train')
model.plotFilter(where='first', data_type='chroma', when='before_train')
model.plotFilter(where='middle', data_type='chroma', when='before_train')
model.plotFilter(where='last', data_type='chroma', when='before_train')

In [ ]:
model.plotFeatureMap(dataset[0][0][0], data_type='spec', when='before_train')
model.plotFeatureMap(dataset[0][0][1], data_type='mfcc', when='before_train')
model.plotFeatureMap(dataset[0][0][2], data_type='chroma', when='before_train')

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
model.train_(train_loader, test_loader, best_lr, 100, device)

In [ ]:
model.plot('train_acc')
model.plot('train_loss')
model.plot('val_acc')
model.plot('val_loss')
model.plot('confusion_train')
model.plot('confusion_normalize_train')
model.plot('confusion_val')
model.plot('confusion_normalize_val')

In [ ]:
spec_weights, spec_layers, mfcc_weights, mfcc_layers, chroma_weights, chroma_layers = model.extractConvLayer()

In [ ]:
model.plotFilter(where='first', data_type='spec', when='after_train')
model.plotFilter(where='middle', data_type='spec', when='after_train')
model.plotFilter(where='last', data_type='spec', when='after_train')
model.plotFilter(where='first', data_type='mfcc', when='after_train')
model.plotFilter(where='middle', data_type='mfcc', when='after_train')
model.plotFilter(where='last', data_type='mfcc', when='after_train')
model.plotFilter(where='first', data_type='chroma', when='after_train')
model.plotFilter(where='middle', data_type='chroma', when='after_train')
model.plotFilter(where='last', data_type='chroma', when='after_train')

In [ ]:
model.plotFeatureMap(dataset[0][0][0].to(device), data_type='spec', when='after_train')
model.plotFeatureMap(dataset[0][0][1].to(device), data_type='mfcc', when='after_train')
model.plotFeatureMap(dataset[0][0][2].to(device), data_type='chroma', when='after_train')